In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from instarecog.facenet import FaceNet
from online_triplet_loss.losses import *

In [8]:
def pretty_log(metastr):
    print(metastr, end='\r')

In [2]:
learning_rate = 0.001
batch_size = 64
embedding_size = 128
num_epochs = 15
margin = 0.5


In [3]:
# Define a transform to preprocess the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to (224, 224)
    transforms.ToTensor(),         # Convert the image to a PyTorch tensor
    transforms.Normalize(          # Normalize the image
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

# Load the LFW dataset
lfw_dataset = torchvision.datasets.ImageFolder(root='../lfw', transform=transform)

# Define a DataLoader to load the dataset in batches
lfw_loader = DataLoader(lfw_dataset, batch_size=batch_size, shuffle=True)

model = FaceNet(embedding_size)

# Define the optimizer
# You can use the Adam optimizer or another suitable optimizer for training the model
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train the model
for epoch in range(num_epochs):
    for batch_idx, (data, labels) in enumerate(lfw_loader):
        pretty_log(f"On epoch {epoch} / {num_epochs}, batch {batch_idx} / {len(lfw_loader)}")
        optimizer.zero_grad()
        embeddings = model(data)
        loss = batch_hard_triplet_loss(labels, embeddings, margin=margin)
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 